## Manipulating and transforming DataFrames

In [1]:
import warnings

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display
from openbb import obb

In [3]:
warnings.filterwarnings("ignore")
obb.user.preferences.output_type = "dataframe"

Fetches historical price data for the equity "AAPL" using the "yfinance" provider and stores it in 'asset'

In [4]:
asset = obb.equity.price.historical("AAPL", provider="yfinance")

In [5]:
display(asset)

,open,high,low,close,volume,split_ratio,dividend
date,,,,,,,
2023-06-15,183.960007,186.520004,183.779999,186.009995,65433200,0.0,0.0
2023-06-16,186.729996,186.990005,184.270004,184.919998,101235600,0.0,0.0
2023-06-20,184.410004,186.100006,184.410004,185.009995,49799100,0.0,0.0
2023-06-21,184.899994,185.410004,182.589996,183.960007,49515700,0.0,0.0
2023-06-22,183.740005,187.050003,183.669998,187.000000,51245300,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-10,196.899994,197.300003,192.149994,193.119995,97262100,0.0,0.0
2024-06-11,193.649994,207.160004,193.630005,207.149994,172373300,0.0,0.0
2024-06-12,207.369995,220.199997,206.899994,213.070007,198134300,0.0,0.0


Fetches historical price data for the equity "SPY" using the "yfinance" provider and stores it in 'benchmark'

In [6]:
benchmark = obb.equity.price.historical("SPY", provider="yfinance")

In [7]:
display(benchmark)

,open,high,low,close,volume,split_ratio,dividend,capital_gains
date,,,,,,,,
2023-06-15,436.329987,443.899994,436.230011,442.600006,110303100,0.0,0.000,0.0
2023-06-16,443.019989,443.609985,438.970001,439.459991,114121300,0.0,1.638,0.0
2023-06-20,437.450012,438.369995,435.029999,437.179993,76160400,0.0,0.000,0.0
2023-06-21,436.160004,436.989990,434.329987,434.940002,76982300,0.0,0.000,0.0
2023-06-22,433.950012,436.619995,433.600006,436.510010,70637200,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...
2024-06-10,533.179993,535.989990,532.570007,535.659973,35729300,0.0,0.000,0.0
2024-06-11,534.070007,537.010010,532.049988,536.950012,36383400,0.0,0.000,0.0
2024-06-12,541.630005,544.119995,540.299988,541.359985,63251300,0.0,0.000,0.0


Updates the column names for 'asset' and 'benchmark'

In [8]:
columns = [
    "open",
    "high",
    "low",
    "close",
    "volume",
    "dividends",
    "splits",
]

In [9]:
asset.columns = columns
benchmark.columns = columns + ["capital_gain"]

Adds a new column 'price_diff' that contains the difference in closing prices

In [10]:
asset["price_diff"] = asset.close.diff()
benchmark["price_diff"] = benchmark.close.diff()

Adds a new column 'gain' that indicates whether the price difference is positive

In [11]:
asset["gain"] = asset.price_diff > 0
benchmark["gain"] = benchmark.price_diff > 0

Adds a new column 'symbol' with the equity symbol

In [12]:
asset["symbol"] = "AAPL"
benchmark["symbol"] = "SPY"

Sets the volume of the 10th row to the mean volume of rows 5 to 9 in 'asset_2'

In [13]:
asset_2 = asset.copy()
asset_2.at[asset_2.index[10], "volume"] = asset_2.volume[asset_2.index[5:10]].mean()

In [14]:
asset_2.iat[10, 5]

0.0

Concatenates 'asset' and 'asset_2' and drops duplicates

In [15]:
df = pd.concat([asset, asset_2]).drop_duplicates()

Adds a new column 'returns' that contains the percentage change in closing prices

In [16]:
asset["returns"] = asset["close"].pct_change()

Creates a pivot table from 'asset' with 'returns' as values, 'gain' as columns, and 'sum', 'mean', 'std' as aggregation functions

In [17]:
df = pd.pivot_table(
    data=asset, values="returns", columns="gain", aggfunc=["sum", "mean", "std"]
)

In [18]:
display(df)

sum                mean                 std          
gain        False     True      False     True      False     True 
returns -1.167966  1.324295 -0.009815  0.010033  0.008522  0.010122

Concatenates 'asset' and 'benchmark' and groups by 'symbol' to calculate open, high, low, close (OHLC) prices

In [19]:
concated = pd.concat([asset, benchmark])

In [20]:
df = concated.groupby("symbol").close.ohlc()

In [21]:
display(df)

,open,high,low,close
symbol,,,,
AAPL,186.009995,214.240005,165.000000,212.490005
SPY,442.600006,542.780029,410.679993,542.780029


Fetches options chains for the symbol "AAPL" using the "cboe" provider and stores it in 'chains'

In [22]:
chains = obb.derivatives.options.chains("AAPL", provider="cboe")

In [23]:
display(chains)

,contract_symbol,expiration,strike,option_type,open_interest,volume,theoretical_price,last_trade_price,tick,bid,...,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho,last_trade_timestamp,dte
0,AAPL240614C00100000,2024-06-14,100.0,call,25,9,112.4902,111.90,down,111.20,...,-2.400,-0.0210,0.0000,1.0000,0.0000,-0.0002,0.0000,0.0000,2024-06-14 14:22:43,-1
1,AAPL240614P00100000,2024-06-14,100.0,put,59,0,0.0002,0.01,no_change,0.00,...,0.000,0.0000,0.0000,0.0000,0.0000,-0.0002,0.0000,0.0000,2024-06-12 12:51:23,-1
2,AAPL240614C00105000,2024-06-14,105.0,call,5,0,107.4902,94.83,up,106.35,...,0.000,0.0000,0.0000,1.0000,0.0000,-0.0002,0.0000,0.0000,2024-06-11 10:02:16,-1
3,AAPL240614P00105000,2024-06-14,105.0,put,3,0,0.0002,0.01,up,0.00,...,0.000,0.0000,0.0000,0.0000,0.0000,-0.0002,0.0000,0.0000,2024-06-12 12:51:23,-1
4,AAPL240614C00110000,2024-06-14,110.0,call,55,1,102.4903,101.47,down,101.20,...,-2.705,-0.0260,0.0000,1.0000,0.0000,-0.0003,0.0000,0.0000,2024-06-14 12:34:35,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,AAPL261218P00330000,2026-12-18,330.0,put,0,0,117.5100,134.83,no_change,115.65,...,0.000,0.0000,0.2921,-1.0000,0.0000,-0.0261,0.0000,0.0000,2024-06-06 15:39:43,916
2178,AAPL261218C00340000,2026-12-18,340.0,call,191,60,6.8216,6.70,down,6.55,...,-0.700,-0.0946,0.2208,0.1980,0.0038,-0.0101,0.9423,0.8726,2024-06-14 13:09:53,916
2179,AAPL261218P00340000,2026-12-18,340.0,put,0,10,127.5100,129.55,up,125.55,...,3.675,0.0292,0.0000,-1.0000,0.0000,-0.0261,0.0000,0.0000,2024-06-14 15:35:37,916
2180,AAPL261218C00350000,2026-12-18,350.0,call,2493,159,5.9017,5.80,down,5.70,...,-0.700,-0.1077,0.2216,0.1762,0.0035,-0.0095,0.8786,0.7790,2024-06-14 15:45:53,916


Extracts unique expiration dates from 'chains'

In [24]:
expirations = chains.expiration.unique()

Filters 'chains' to get call options expiring on the 6th expiration date and stores it in 'calls'

In [25]:
calls = chains[(chains.option_type == "call") & (chains.expiration == expirations[5])]

Filters 'chains' to get put options expiring on the 6th expiration date and stores it in 'puts'

In [26]:
puts = chains[(chains.option_type == "put") & (chains.expiration == expirations[5])]

Sets the index of 'calls' and 'puts' to 'strike'

In [27]:
calls_strike = calls.set_index("strike")
puts_strike = puts.set_index("strike")

Joins 'calls_strike' and 'puts_strike' on the 'strike' index with a left join, adding suffixes to overlapping column names

In [28]:
joined = calls_strike.join(puts_strike, how="left", lsuffix="_call", rsuffix="_put")

Extracts the last trade prices of call and put options and stores them in 'prices'

In [29]:
prices = joined[["last_trade_price_call", "last_trade_price_put"]]

In [30]:
display(prices)

,last_trade_price_call,last_trade_price_put
strike,,
5.0,187.72,0.00
10.0,169.81,0.01
15.0,179.20,0.01
20.0,0.00,0.01
25.0,155.00,0.01
30.0,162.45,0.00
35.0,156.48,0.01
40.0,0.00,0.01
45.0,0.00,0.00


Adds a new column 'straddle_price' that contains the sum of last trade prices of call and put options

In [31]:
prices.loc[:, "straddle_price"] = prices.sum(axis=1)

In [32]:
display(prices)

,last_trade_price_call,last_trade_price_put,straddle_price
strike,,,
5.0,187.72,0.00,187.72
10.0,169.81,0.01,169.82
15.0,179.20,0.01,179.21
20.0,0.00,0.01,0.01
25.0,155.00,0.01,155.01
30.0,162.45,0.00,162.45
35.0,156.48,0.01,156.49
40.0,0.00,0.01,0.01
45.0,0.00,0.00,0.00


Groups 'chains' by option type, strike, and expiration to calculate the sum of open interest

In [33]:
df = chains.groupby(["option_type", "strike", "expiration"]).open_interest.sum()

In [34]:
display(df)

option_type  strike  expiration
call         5.0     2024-06-21    82
                     2024-07-19    12
                     2024-08-16     1
                     2024-09-20     5
                     2024-10-18     0
                                   ..
put          370.0   2024-08-16     0
                     2024-10-18     0
                     2024-11-15     0
                     2025-03-21     0
             380.0   2025-03-21     0
Name: open_interest, Length: 2182, dtype: int64

Groups 'chains' by option type, strike, and expiration to calculate the max last trade price and the sum of open interest

In [35]:
df = chains.groupby(["option_type", "strike", "expiration"]).agg(
    {"last_trade_price": "max", "open_interest": "sum"}
)

In [36]:
display(df)

last_trade_price  open_interest
option_type strike expiration                                 
call        5.0    2024-06-21            186.82             82
                   2024-07-19            187.72             12
                   2024-08-16            182.23              1
                   2024-09-20            208.94              5
                   2024-10-18            172.06              0
...                                         ...            ...
put         370.0  2024-08-16              0.00              0
                   2024-10-18              0.00              0
                   2024-11-15            187.00              0
                   2025-03-21              0.00              0
            380.0  2025-03-21            189.89              0

[2182 rows x 2 columns]

Groups 'chains' by option type and calculates the mean bid-ask spread for each group

In [37]:
df = chains.groupby(["option_type"]).apply(
    lambda x: (x["ask"] - x["bid"]).mean(), include_groups=False
)

In [38]:
display(df)

option_type
call    0.963648
put     0.663355
dtype: float64

Standardizes the last trade prices by subtracting the mean and dividing by the standard deviation within each expiration group

In [39]:
(
    chains.groupby("expiration").last_trade_price.transform(
        lambda x: (x - x.mean()) / x.std()
    )
)

0       3.267720
1      -0.677929
2       2.665769
3      -0.677929
4       2.899920
          ...   
2177    1.758217
2178   -0.860451
2179    1.650306
2180   -0.878845
2181    2.387694
Name: last_trade_price, Length: 2182, dtype: float64

In [40]:
display(chains)

,contract_symbol,expiration,strike,option_type,open_interest,volume,theoretical_price,last_trade_price,tick,bid,...,change,change_percent,implied_volatility,delta,gamma,theta,vega,rho,last_trade_timestamp,dte
0,AAPL240614C00100000,2024-06-14,100.0,call,25,9,112.4902,111.90,down,111.20,...,-2.400,-0.0210,0.0000,1.0000,0.0000,-0.0002,0.0000,0.0000,2024-06-14 14:22:43,-1
1,AAPL240614P00100000,2024-06-14,100.0,put,59,0,0.0002,0.01,no_change,0.00,...,0.000,0.0000,0.0000,0.0000,0.0000,-0.0002,0.0000,0.0000,2024-06-12 12:51:23,-1
2,AAPL240614C00105000,2024-06-14,105.0,call,5,0,107.4902,94.83,up,106.35,...,0.000,0.0000,0.0000,1.0000,0.0000,-0.0002,0.0000,0.0000,2024-06-11 10:02:16,-1
3,AAPL240614P00105000,2024-06-14,105.0,put,3,0,0.0002,0.01,up,0.00,...,0.000,0.0000,0.0000,0.0000,0.0000,-0.0002,0.0000,0.0000,2024-06-12 12:51:23,-1
4,AAPL240614C00110000,2024-06-14,110.0,call,55,1,102.4903,101.47,down,101.20,...,-2.705,-0.0260,0.0000,1.0000,0.0000,-0.0003,0.0000,0.0000,2024-06-14 12:34:35,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,AAPL261218P00330000,2026-12-18,330.0,put,0,0,117.5100,134.83,no_change,115.65,...,0.000,0.0000,0.2921,-1.0000,0.0000,-0.0261,0.0000,0.0000,2024-06-06 15:39:43,916
2178,AAPL261218C00340000,2026-12-18,340.0,call,191,60,6.8216,6.70,down,6.55,...,-0.700,-0.0946,0.2208,0.1980,0.0038,-0.0101,0.9423,0.8726,2024-06-14 13:09:53,916
2179,AAPL261218P00340000,2026-12-18,340.0,put,0,10,127.5100,129.55,up,125.55,...,3.675,0.0292,0.0000,-1.0000,0.0000,-0.0261,0.0000,0.0000,2024-06-14 15:35:37,916
2180,AAPL261218C00350000,2026-12-18,350.0,call,2493,159,5.9017,5.80,down,5.70,...,-0.700,-0.1077,0.2216,0.1762,0.0035,-0.0095,0.8786,0.7790,2024-06-14 15:45:53,916


**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.